여기부터 하면 됨

In [93]:
import pandas as pd

store = pd.read_csv("../dataset/store2.csv", encoding='utf-8')
# pd.set_option("display.max_rows", None)
store.업태명.value_counts()

업태명
한식                 2507
기타                 1107
커피숍                 937
호프/통닭               590
분식                  519
일반조리판매              361
편의점                 343
기타 휴게음식점            324
일식                  286
경양식                 275
정종/대포집/소주방          257
중국식                 239
통닭(치킨)              209
패스트푸드               144
식육(숯불구이)            100
까페                   88
횟집                   70
외국음식전문점(인도,태국등)      36
뷔페식                  30
아이스크림                20
키즈카페                 13
다방                   12
김밥(도시락)              11
탕류(보신용)              10
과자점                   8
복어취급                  6
출장조리                  5
패밀리레스트랑               5
라이브카페                 4
떡카페                   4
냉면집                   2
백화점                   2
철도역구내                 2
푸드트럭                  2
Name: count, dtype: int64

In [94]:
drop_conditions = ['편의점', '푸드트럭', '출장조리', '라이브카페']
print(f"삭제 이전 : {len(store)}행")

store = store[~store.업태명.isin(drop_conditions)]
print(f"삭제 이후 : {len(store)}행")

store.업태명.value_counts()

삭제 이전 : 8528행
삭제 이후 : 8174행


업태명
한식                 2507
기타                 1107
커피숍                 937
호프/통닭               590
분식                  519
일반조리판매              361
기타 휴게음식점            324
일식                  286
경양식                 275
정종/대포집/소주방          257
중국식                 239
통닭(치킨)              209
패스트푸드               144
식육(숯불구이)            100
까페                   88
횟집                   70
외국음식전문점(인도,태국등)      36
뷔페식                  30
아이스크림                20
키즈카페                 13
다방                   12
김밥(도시락)              11
탕류(보신용)              10
과자점                   8
복어취급                  6
패밀리레스트랑               5
떡카페                   4
냉면집                   2
백화점                   2
철도역구내                 2
Name: count, dtype: int64

In [2]:
import time
import warnings
warnings.filterwarnings('ignore')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup

In [77]:

# store.loc[store.addr == "서울특별시 동작구 흑석동 223-1 ", '업소명']

sample = store[:100]
sample.tail()

,업종명,인허가일자,업소명,소재지(도로명),addr,영업장면적(건물내_외부),업태명,담당부서,데이터기준일자,latitude,longitude
95,일반음식점,1989-08-17,불난떡볶이 노량진점,서울특별시 동작구 노량진로8길 36 (노량진동),서울특별시 동작구 노량진동 258-40,27.17,한식,보건행정과,2024-08-19,37.512459,126.937252
96,일반음식점,1989-11-07,밀숲노량진역점,서울특별시 동작구 만양로14길 6-1 (노량진동),서울특별시 동작구 노량진동 123-8,40.56,호프/통닭,보건행정과,2024-08-19,37.511337,126.944868
97,일반음식점,1989-06-19,한강호프,NaN,서울특별시 동작구 흑석동 33-91,24.30,정종/대포집/소주방,보건행정과,2024-08-19,37.505836,126.969822
98,일반음식점,1989-01-13,뚝배기 양평해장국 신대방점,"서울특별시 동작구 보라매로 99, 1층 (신대방동)",서울특별시 동작구 신대방동 350-9,60.66,식육(숯불구이),보건행정과,2024-08-19,37.498165,126.927781
99,일반음식점,1989-03-17,원조꼬끼오숯불통닭,서울특별시 동작구 서달로10길 6 (흑석동),서울특별시 동작구 흑석동 86-74,22.44,통닭(치킨),보건행정과,2024-08-19,37.506435,126.961666


In [79]:
driver = webdriver.Chrome()

# 카카오 맵으로 이동
url = "https://map.kakao.com/"
driver.get(url)
sample = store[50:100]
rest_list = []

for store_name, addr in zip(sample["업소명"], sample["addr"]):
    # 로고 클릭해 초기화 -> 검색창 clear로 대체
    # driver.find_element(By.XPATH, '/html/body/div[2]/div/h1/a').send_keys(Keys.ENTER)
    
    # 카카오맵 검색창
    search_area = driver.find_element(By.ID, "search.keyword.query")
    # 기존 검색어 제거
    search_area.clear()
    # 검색어 전달
    search_area.send_keys(addr) 
    
    # 돋보기 클릭(검색)
    driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
    
    time.sleep(2)
    try:
        num_of_result = int(driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/div[1]/span/em').text)
        num_of_result = min(5, num_of_result)
        
        # 결과 0건이면 이후 패스
        if num_of_result == 0:
            continue
            
        same_list = store.loc[store.addr == addr, '업소명']
        # 결과창 A~E까지 받아옴
        for t in range(1,num_of_result+1):
            xpath_name = f'/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/ul/li[{t}]/div[3]/strong/a[2]'
            name = driver.find_element(By.XPATH, xpath_name).text
    
            try:
                xpath_review =  f'/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/ul/li[{t}]/div[4]/span[1]/a'
                review_num = int(driver.find_element(By.XPATH, xpath_review).text[:-1])
    
                if name == store_name and review_num >= 2:
                    print(f"{name}: 음식점 저장")
                    rest_list.append(name)
                    break

            except:
                pass
    
    except:
        pass
print(rest_list)

성상등갈비: 음식점 저장
잠꾸러기: 음식점 저장
연경: 음식점 저장
꿀꾸랄라: 음식점 저장
향백: 음식점 저장
성림식당: 음식점 저장
연래춘: 음식점 저장
태원: 음식점 저장
우림: 음식점 저장
부자국밥: 음식점 저장
곽반장: 음식점 저장
백제갈비: 음식점 저장
다올집: 음식점 저장
남도미가: 음식점 저장
소품은돼지: 음식점 저장
불난떡볶이 노량진점: 음식점 저장
['성상등갈비', '잠꾸러기', '연경', '꿀꾸랄라', '향백', '성림식당', '연래춘', '태원', '우림', '부자국밥', '곽반장', '백제갈비', '다올집', '남도미가', '소품은돼지', '불난떡볶이 노량진점']


In [46]:
rest_list

[]

In [102]:
# /html/body/div[2]/div[2]/div[2]/div[8]/div[3]/a/span[1]

     업종명       인허가일자         업소명                      소재지(도로명)  \
0  일반음식점  1967-10-10  케이에프씨 중앙대점        서울특별시 동작구 흑석로 94 (흑석동)   
1  일반음식점  1972-04-14         안동장     서울특별시 동작구 흑석로 105-1 (흑석동)   
2  일반음식점  1975-05-15       바다회센타  서울특별시 동작구 동작대로27길 20-7 (사당동)   
3  일반음식점  1975-06-17        신부양각       서울특별시 동작구 양녕로 191 (상도동)   
4  일반음식점  1976-02-16        순대나라     서울특별시 동작구 서달로12가길 5 (흑석동)   

                     addr  영업장면적(건물내_외부)  업태명   담당부서     데이터기준일자   latitude  \
0    서울특별시 동작구 흑석동 223-1          163.96   기타  보건행정과  2024-08-19  37.507262   
1   서울특별시 동작구 흑석동 184-18          321.03  중국식  보건행정과  2024-08-19  37.508251   
2   서울특별시 동작구 사당동 130-16           71.41   일식  보건행정과  2024-08-19  37.487893   
3  서울특별시 동작구 상도동 210-202           91.84  중국식  보건행정과  2024-08-19  37.498381   
4    서울특별시 동작구 흑석동 97-22           25.50   한식  보건행정과  2024-08-19  37.506608   

    longitude  
0  126.959988  
1  126.960935  
2  126.980722  
3  126.943482  
4  126.962693  


In [112]:
driver = webdriver.Chrome()

# 카카오 맵으로 이동
url = "https://map.kakao.com/"
driver.get(url)
sample = store[:10]
rest_list = []

for store_name, addr in zip(sample["업소명"], sample["addr"]):    
    # 카카오맵 검색창
    search_area = driver.find_element(By.ID, "search.keyword.query")
    # 기존 검색어 제거
    search_area.clear()
    # 검색어 전달
    search_area.send_keys(addr) 
    
    # 돋보기 클릭(검색)
    driver.find_element(By.XPATH, r'//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)
    
    time.sleep(2)
    
    num_of_result = int(driver.find_element(By.XPATH, '/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/div[1]/span/em').text)
    num_of_result = min(5, num_of_result)
    
    # 결과 0건이면 이후 패스
    if num_of_result == 0:
        continue
        
    same_list = store.loc[store.addr == addr, '업소명']
    # 결과창 A~E까지 받아옴
    for t in range(1,num_of_result+1):
        xpath_name = f'/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/ul/li[{t}]/div[3]/strong/a[2]'
        name = driver.find_element(By.XPATH, xpath_name).text
        
        temp = dict()
        try:
            xpath_review =  f'/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/ul/li[{t}]/div[4]/span[1]/a'
            review_num = int(driver.find_element(By.XPATH, xpath_review).text[:-1])
            
            xpath_score = f'/html/body/div[5]/div[2]/div[1]/div[7]/div[5]/ul/li[{t}]/div[4]/span[1]/em'
            score = float(driver.find_element(By.XPATH, xpath_score).text)

            if name == store_name and review_num >= 5:
                print(f"{name}: 음식점 저장 --- {score}점(리뷰 {review_num}개)")

                temp["store_name"] = store_name
                temp["addr"] = addr
                temp["score"] = score
                
                ### 리뷰 크롤링
                # 리뷰 건수를 눌러서 리뷰 탭으로 이동
                driver.find_element(By.XPATH, xpath_review).send_keys(Keys.ENTER)
                # driver.switch_to.window(driver.window_handles[-1])
                time.sleep(10)
                
        except:
            # 다시 검색창으로 이동
            driver.switch_to.window(driver.window_handles[0])
            time.sleep(2)
            pass

안동장: 음식점 저장 --- 2.4점(리뷰 55개)
신부양각: 음식점 저장 --- 4.4점(리뷰 9개)
순대나라: 음식점 저장 --- 4.4점(리뷰 39개)
우슬재: 음식점 저장 --- 4.4점(리뷰 7개)


In [33]:
def save_reviews(url):
    """
    :param url: 리뷰 추출할 웹 사이트 주소
    :return: 리뷰들 리스트
    """
    review_dict = {}
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)
    try:    
        while True:
            # 후기 더보기 버튼이 있으면 계속 반복
            try:
                txt_more = driver.find_element(By.XPATH, '//span[@class="txt_more" and contains(text(), "후기 더보기")]')
                link_more_button = txt_more.find_element(By.XPATH, './ancestor::a[contains(@class, "link_more")]')
                time.sleep(.5)
                
                if link_more_button and link_more_button.text != "후기 접기":
                    link_more_button.click()
                    # print("후기 더보기 클릭")
            
            # 없으면 탈출
            except:
                break

    except:
        print("후기 더보기 버튼 없음")
    
    # 리뷰들 긁어오기 
    reviews = driver.find_elements(By.CLASS_NAME, 'txt_comment')
    for review in reviews:
        # 더보기 죽이기
        txt = review.text.replace("더보기","")
        review_dict.add(txt)
        
    return review_dict

In [38]:
# driver = webdriver.Chrome()
# url = "https://place.map.kakao.com/929624344"             # 리뷰 많음
# # url = "https://place.map.kakao.com/1841716590#comment"      # 카츠진
# # url = "https://place.map.kakao.com/22455352"            # 리뷰 3개
# driver.get(url)
# time.sleep(2)
# txt_more = None

test_dict = save_reviews("https://place.map.kakao.com/22455352")
print(test_dict)
for text in test_dict:
    print(text)

    






['양다리는 30분전에만 예약하면 된다셔서 가기전에 예약하고 가니 딱 맞게 세팅해놓으셨어요\n2인이였는데 8만원짜리 나왔구 엄청 배부르게먹고 포장해서 다음날까지 먹었어요\n3인이서 시키시면 알맞을거같아요\n\n불 거의 나갈때쯤엔 사장님께서 썰어주시고 포장도 가능해요~! 서비스로 만두도 주셨는데 만두는 만두피가 두꺼워서 별로요\n반찬으로나오는 면두부 무침 맛있어요\n', '', '역시 숨겨진 맛집이라 리뷰가 없네요\n중국손님들이 많고 장소가 협소합니다만\n사장님부부가 친절하고 양다리도 맛있어요\n한국손님위주가 장사 아니라서 많은 기대하면 x\n손님이 많아서  저녁시간엔 예약안하면 힘듭니다 \n양다리는 최소한 3시간 전에 예약필수\n']
양다리는 30분전에만 예약하면 된다셔서 가기전에 예약하고 가니 딱 맞게 세팅해놓으셨어요
2인이였는데 8만원짜리 나왔구 엄청 배부르게먹고 포장해서 다음날까지 먹었어요
3인이서 시키시면 알맞을거같아요

불 거의 나갈때쯤엔 사장님께서 썰어주시고 포장도 가능해요~! 서비스로 만두도 주셨는데 만두는 만두피가 두꺼워서 별로요
반찬으로나오는 면두부 무침 맛있어요


역시 숨겨진 맛집이라 리뷰가 없네요
중국손님들이 많고 장소가 협소합니다만
사장님부부가 친절하고 양다리도 맛있어요
한국손님위주가 장사 아니라서 많은 기대하면 x
손님이 많아서  저녁시간엔 예약안하면 힘듭니다 
양다리는 최소한 3시간 전에 예약필수



In [19]:
rest_list = []